# Time Series

## 1. Importing libraries and files

In [24]:
import numpy as np 
import pandas as pd

In [25]:
df_weekly = pd.read_csv('./Data_for_Power_BI/weekly_data_for_power_bi.csv')

In [26]:
df_cluster = pd.read_csv('./df_cluster.csv')

In [27]:
path_events = './Data_for_Power_BI/daily_calendar_with_expanded_events.csv'
df_calendar_original = pd.read_csv(path_events)

In [81]:
df_calendar = df_calendar_original.groupby('yearweek').agg({'date': 'min', 'Weekly_Event' : 'first'}).reset_index()
df_calendar['Year'] = pd.to_datetime(df_calendar['date']).dt.year
df_calendar['Quarter'] = pd.to_datetime(df_calendar['date']).dt.quarter
df_calendar['Month'] = pd.to_datetime(df_calendar['date']).dt.month
df_calendar['Day_month'] = pd.to_datetime(df_calendar['date']).dt.day

In [82]:
df_calendar.head()

,yearweek,date,Weekly_Event,Year,Quarter,Month,Day_month
0,201105.0,2011-01-29,No Event,2011,1,1,29
1,201106.0,2011-02-05,Superbowl,2011,1,2,5
2,201107.0,2011-02-12,Valentine's Day,2011,1,2,12
3,201108.0,2011-02-19,Presidents' Day,2011,1,2,19
4,201109.0,2011-02-26,No Event,2011,1,2,26


In [30]:
df_weekly.head()

,item,store_code,category_x,department,region,store,yearweek,sale,sell_price,daily_revenue,week_number
0,ACCESORIES_1_004,BOS_1,ACCESORIES,ACCESORIES_1,Boston,South_End,201105.0,10,5.77,57.72,1
1,ACCESORIES_1_004,BOS_2,ACCESORIES,ACCESORIES_1,Boston,Roxbury,201105.0,15,5.77,86.58,1
2,ACCESORIES_1_004,NYC_2,ACCESORIES,ACCESORIES_1,New York,Harlem,201105.0,33,5.77,190.48,1
3,ACCESORIES_1_004,NYC_4,ACCESORIES,ACCESORIES_1,New York,Brooklyn,201105.0,8,5.77,46.18,1
4,ACCESORIES_1_004,PHI_1,ACCESORIES,ACCESORIES_1,Philadelphia,Midtown_Village,201105.0,6,5.77,34.63,1


In [31]:
df_weekly.shape

(6721786, 11)

In [32]:
df_cluster.head()

,sales_ratio_mean,sell_price_mean,sales_ratio_relative_std,release_week,Sociability,Regionality,Seasonality,%trend_slope,price_elasticity,cluster,item
0,2.760999,11.043445,0.290107,131,0.0,0.400899,0.150287,0.0,-0.330625,5,ACCESORIES_1_001
1,1.855793,5.275790,0.582964,6,0.0,0.366412,0.127449,0.0,-0.059177,4,ACCESORIES_1_002
2,1.252894,3.943779,0.565078,161,0.0,0.178200,0.104309,0.0,0.029035,0,ACCESORIES_1_003
3,14.145769,5.981574,0.228632,1,1.0,0.315894,0.030596,0.0,0.308701,4,ACCESORIES_1_004
4,5.570844,3.832410,0.422320,14,0.0,0.212243,0.097256,0.0,-0.369176,1,ACCESORIES_1_005


In [33]:
df_cluster.shape

(3049, 11)

In [34]:
df_calendar.head()

,yearweek,date,Year,Quarter,Month,Day_month
0,201105.0,2011-01-29,2011,1,1,29
1,201106.0,2011-02-05,2011,1,2,5
2,201107.0,2011-02-12,2011,1,2,12
3,201108.0,2011-02-19,2011,1,2,19
4,201109.0,2011-02-26,2011,1,2,26


In [35]:
df_calendar.shape

(279, 6)

In [36]:
df_weekly.week_number.max()

279

In [37]:
df_weekly[df_weekly.week_number > 104].shape

(4886761, 11)

In [38]:
df_weekly[df_weekly.week_number > 278]['yearweek'].unique()

array([201617.])

In [39]:
# Merge calendar features into weekly dataframe
df_weekly = df_weekly.merge(df_calendar[['yearweek', 'Year', 'Quarter', 'Month', 'Day_month']], on='yearweek', how='left')

In [40]:
df_weekly.head()

,item,store_code,category_x,department,region,store,yearweek,sale,sell_price,daily_revenue,week_number,Year,Quarter,Month,Day_month
0,ACCESORIES_1_004,BOS_1,ACCESORIES,ACCESORIES_1,Boston,South_End,201105.0,10,5.77,57.72,1,2011,1,1,29
1,ACCESORIES_1_004,BOS_2,ACCESORIES,ACCESORIES_1,Boston,Roxbury,201105.0,15,5.77,86.58,1,2011,1,1,29
2,ACCESORIES_1_004,NYC_2,ACCESORIES,ACCESORIES_1,New York,Harlem,201105.0,33,5.77,190.48,1,2011,1,1,29
3,ACCESORIES_1_004,NYC_4,ACCESORIES,ACCESORIES_1,New York,Brooklyn,201105.0,8,5.77,46.18,1,2011,1,1,29
4,ACCESORIES_1_004,PHI_1,ACCESORIES,ACCESORIES_1,Philadelphia,Midtown_Village,201105.0,6,5.77,34.63,1,2011,1,1,29


## 2. Feature Engineering

In [41]:
df_weekly.columns

Index(['item', 'store_code', 'category_x', 'department', 'region', 'store',
       'yearweek', 'sale', 'sell_price', 'daily_revenue', 'week_number',
       'Year', 'Quarter', 'Month', 'Day_month'],
      dtype='object')

In [42]:
df_price_mean = df_weekly.groupby(['Year', 'item']).agg({'sell_price': 'mean', 'department' : 'first'}).reset_index()
df_price_mean.rename(columns={'sell_price': 'price_mean_year'}, inplace=True)
df_price_mean.head()

,Year,item,price_mean_year,department
0,2011,ACCESORIES_1_002,5.280000,ACCESORIES_1
1,2011,ACCESORIES_1_004,5.693054,ACCESORIES_1
2,2011,ACCESORIES_1_005,3.610588,ACCESORIES_1
3,2011,ACCESORIES_1_008,0.641891,ACCESORIES_1
4,2011,ACCESORIES_1_009,2.251861,ACCESORIES_1


In [43]:
df_price_depart = df_price_mean.groupby(['Year', 'department']).agg({'price_mean_year': 'mean'}).reset_index()
df_price_depart.rename(columns={'price_mean_year': 'price_mean_department_year'}, inplace=True)
df_price_depart.head()

,Year,department,price_mean_department_year
0,2011,ACCESORIES_1,7.001967
1,2011,ACCESORIES_2,3.733769
2,2011,HOME_&_GARDEN_1,6.085186
3,2011,HOME_&_GARDEN_2,7.343540
4,2011,SUPERMARKET_1,4.203097


In [44]:
df_prices_aggregated = df_price_mean.merge(df_price_depart, on=['Year', 'department'], how='left')
df_prices_aggregated.head()

,Year,item,price_mean_year,department,price_mean_department_year
0,2011,ACCESORIES_1_002,5.280000,ACCESORIES_1,7.001967
1,2011,ACCESORIES_1_004,5.693054,ACCESORIES_1,7.001967
2,2011,ACCESORIES_1_005,3.610588,ACCESORIES_1,7.001967
3,2011,ACCESORIES_1_008,0.641891,ACCESORIES_1,7.001967
4,2011,ACCESORIES_1_009,2.251861,ACCESORIES_1,7.001967


In [45]:
df_prices_aggregated['price_proportion_item_department'] = df_prices_aggregated['price_mean_year'] / df_prices_aggregated['price_mean_department_year']
df_prices_aggregated.head()

,Year,item,price_mean_year,department,price_mean_department_year,price_proportion_item_department
0,2011,ACCESORIES_1_002,5.280000,ACCESORIES_1,7.001967,0.754074
1,2011,ACCESORIES_1_004,5.693054,ACCESORIES_1,7.001967,0.813065
2,2011,ACCESORIES_1_005,3.610588,ACCESORIES_1,7.001967,0.515653
3,2011,ACCESORIES_1_008,0.641891,ACCESORIES_1,7.001967,0.091673
4,2011,ACCESORIES_1_009,2.251861,ACCESORIES_1,7.001967,0.321604


In [46]:
# para agregar los datos de df aggregated al df weekly con un lag de 1 año,
# cambiamos los datos de year por year + 1

df_prices_aggregated['Year_lag1'] = df_prices_aggregated['Year'] + 1

In [47]:
df_weekly_backup1 = df_weekly.copy()

In [48]:
# Añadimos las nuevas features al df_weekly
df_weekly = df_weekly.merge(df_prices_aggregated
                            [['item', 'Year_lag1', 'price_mean_year', 'price_proportion_item_department']],
                            left_on=['item', 'Year'],
                            right_on=['item', 'Year_lag1'],
                            how='left')
df_weekly.head()

,item,store_code,category_x,department,region,store,yearweek,sale,sell_price,daily_revenue,week_number,Year,Quarter,Month,Day_month,Year_lag1,price_mean_year,price_proportion_item_department
0,ACCESORIES_1_004,BOS_1,ACCESORIES,ACCESORIES_1,Boston,South_End,201105.0,10,5.77,57.72,1,2011,1,1,29,NaN,NaN,NaN
1,ACCESORIES_1_004,BOS_2,ACCESORIES,ACCESORIES_1,Boston,Roxbury,201105.0,15,5.77,86.58,1,2011,1,1,29,NaN,NaN,NaN
2,ACCESORIES_1_004,NYC_2,ACCESORIES,ACCESORIES_1,New York,Harlem,201105.0,33,5.77,190.48,1,2011,1,1,29,NaN,NaN,NaN
3,ACCESORIES_1_004,NYC_4,ACCESORIES,ACCESORIES_1,New York,Brooklyn,201105.0,8,5.77,46.18,1,2011,1,1,29,NaN,NaN,NaN
4,ACCESORIES_1_004,PHI_1,ACCESORIES,ACCESORIES_1,Philadelphia,Midtown_Village,201105.0,6,5.77,34.63,1,2011,1,1,29,NaN,NaN,NaN


In [49]:
df_weekly[df_weekly.Year >= 2012].head()

,item,store_code,category_x,department,region,store,yearweek,sale,sell_price,daily_revenue,week_number,Year,Quarter,Month,Day_month,Year_lag1,price_mean_year,price_proportion_item_department
760780,ACCESORIES_1_002,BOS_1,ACCESORIES,ACCESORIES_1,Boston,South_End,201202.0,0,5.28,0.00,51,2012,1,1,7,2012.0,5.28,0.754074
760781,ACCESORIES_1_002,BOS_2,ACCESORIES,ACCESORIES_1,Boston,Roxbury,201202.0,0,5.28,0.00,51,2012,1,1,7,2012.0,5.28,0.754074
760782,ACCESORIES_1_002,BOS_3,ACCESORIES,ACCESORIES_1,Boston,Back_Bay,201202.0,1,5.28,5.28,51,2012,1,1,7,2012.0,5.28,0.754074
760783,ACCESORIES_1_002,NYC_1,ACCESORIES,ACCESORIES_1,New York,Greenwich_Village,201202.0,1,5.28,5.28,51,2012,1,1,7,2012.0,5.28,0.754074
760784,ACCESORIES_1_002,NYC_2,ACCESORIES,ACCESORIES_1,New York,Harlem,201202.0,0,5.28,0.00,51,2012,1,1,7,2012.0,5.28,0.754074


In [50]:
df_weekly.drop(columns=['Year_lag1'], inplace=True)

In [51]:
df_weekly['changed_price_vs_mean'] = (df_weekly['sell_price'] / df_weekly['price_mean_year']) - 1

In [52]:
# Posibles Features

# cluster OK - falta OHE

  # nivel de precio
    # Precio medio - precio medio por año anterior OK
    # Precio articulo / Precio medio department - precio medio dept año anterior OK

  # elasticidad
    # Cambio de precio                        # % (Precio Actual / Precio medio año anterior) - 1  OK

  #tendencia
    # permanencia en el mercado               # number_week - release_week OK
    # lag ventas (52)/(26) 

  #Sociability


  #Regionality - con datos del año anterior
    # Q(NY), Q(B), Q(PHI)

  #Seasonality
    # Q(Q1), Q(Q2), Q(Q3), Q(Q4)

  #Estables
    # media movil (3), (6)

# articulo, semana, y tienda

# preprocesamiento del target (dif, log)

  # id tienda OK
  # id ciudad OK

  # ventas medias por ciudad, por ciudad y categori
  # ventas medias por tiendas
  

  # semana previa a evento
  # semana evento
  # semana post evento
  # Clasificacion ABC Global                # ABC ITEM // PRECIO_ITEM / PRECIO_PROM_GENERAL

  # año OK
  # mes / trimestre OK
  # sin / cos (year) OK
  # inicio / fin de mes OK
  
  # id category OK
  # id deparment OK



  # climaticos: [T° media semanal por ciudad, Precipitacion media]

In [53]:
#Añadimos datos del clustering y de la semana de lanzamiento
df_weekly = df_weekly.merge(df_cluster[['item', 'cluster', 'release_week']], on='item', how='left')

In [54]:
df_weekly.head()

,item,store_code,category_x,department,region,store,yearweek,sale,sell_price,daily_revenue,week_number,Year,Quarter,Month,Day_month,price_mean_year,price_proportion_item_department,changed_price_vs_mean,cluster,release_week
0,ACCESORIES_1_004,BOS_1,ACCESORIES,ACCESORIES_1,Boston,South_End,201105.0,10,5.77,57.72,1,2011,1,1,29,NaN,NaN,NaN,4,1
1,ACCESORIES_1_004,BOS_2,ACCESORIES,ACCESORIES_1,Boston,Roxbury,201105.0,15,5.77,86.58,1,2011,1,1,29,NaN,NaN,NaN,4,1
2,ACCESORIES_1_004,NYC_2,ACCESORIES,ACCESORIES_1,New York,Harlem,201105.0,33,5.77,190.48,1,2011,1,1,29,NaN,NaN,NaN,4,1
3,ACCESORIES_1_004,NYC_4,ACCESORIES,ACCESORIES_1,New York,Brooklyn,201105.0,8,5.77,46.18,1,2011,1,1,29,NaN,NaN,NaN,4,1
4,ACCESORIES_1_004,PHI_1,ACCESORIES,ACCESORIES_1,Philadelphia,Midtown_Village,201105.0,6,5.77,34.63,1,2011,1,1,29,NaN,NaN,NaN,4,1


In [55]:
df_weekly.columns

Index(['item', 'store_code', 'category_x', 'department', 'region', 'store',
       'yearweek', 'sale', 'sell_price', 'daily_revenue', 'week_number',
       'Year', 'Quarter', 'Month', 'Day_month', 'price_mean_year',
       'price_proportion_item_department', 'changed_price_vs_mean', 'cluster',
       'release_week'],
      dtype='object')

In [56]:
#permanencia en el mercado
df_weekly['market_presence'] = df_weekly['week_number'] - df_weekly['release_week']

In [57]:
df_weekly.columns

Index(['item', 'store_code', 'category_x', 'department', 'region', 'store',
       'yearweek', 'sale', 'sell_price', 'daily_revenue', 'week_number',
       'Year', 'Quarter', 'Month', 'Day_month', 'price_mean_year',
       'price_proportion_item_department', 'changed_price_vs_mean', 'cluster',
       'release_week', 'market_presence'],
      dtype='object')

In [58]:
## Seno y coseno del año
df_weekly['sin_year'] = np.sin(2 * np.pi * df_weekly['Month'] / 12)
df_weekly['cos_year'] = np.cos(2 * np.pi * df_weekly['Month'] / 12)
df_weekly.head()

,item,store_code,category_x,department,region,store,yearweek,sale,sell_price,daily_revenue,...,Month,Day_month,price_mean_year,price_proportion_item_department,changed_price_vs_mean,cluster,release_week,market_presence,sin_year,cos_year
0,ACCESORIES_1_004,BOS_1,ACCESORIES,ACCESORIES_1,Boston,South_End,201105.0,10,5.77,57.72,...,1,29,NaN,NaN,NaN,4,1,0,0.5,0.866025
1,ACCESORIES_1_004,BOS_2,ACCESORIES,ACCESORIES_1,Boston,Roxbury,201105.0,15,5.77,86.58,...,1,29,NaN,NaN,NaN,4,1,0,0.5,0.866025
2,ACCESORIES_1_004,NYC_2,ACCESORIES,ACCESORIES_1,New York,Harlem,201105.0,33,5.77,190.48,...,1,29,NaN,NaN,NaN,4,1,0,0.5,0.866025
3,ACCESORIES_1_004,NYC_4,ACCESORIES,ACCESORIES_1,New York,Brooklyn,201105.0,8,5.77,46.18,...,1,29,NaN,NaN,NaN,4,1,0,0.5,0.866025
4,ACCESORIES_1_004,PHI_1,ACCESORIES,ACCESORIES_1,Philadelphia,Midtown_Village,201105.0,6,5.77,34.63,...,1,29,NaN,NaN,NaN,4,1,0,0.5,0.866025


In [59]:
df_weekly[['Month', 'cos_year']].value_counts()

Month  cos_year     
12      1.000000e+00    660088
3       6.123234e-17    640125
4      -5.000000e-01    594577
1       8.660254e-01    573464
8      -5.000000e-01    566746
2       5.000000e-01    559127
11      8.660254e-01    549342
10      5.000000e-01    537768
5      -8.660254e-01    524237
6      -1.000000e+00    515704
9      -1.836970e-16    507913
7      -8.660254e-01    492695
Name: count, dtype: int64

#### Añadimos el dataframe del tiempo meteorológico

In [60]:
from meteostat import Point, Daily
from datetime import datetime, timedelta

In [61]:

# Diccionario con las coordenadas de las estaciones de aeropuerto
cities_config = {
    "Boston": Point(42.3656, -71.0096),       # KBOS
    "Philadelphia": Point(39.8730, -75.2437), # KPHL
    "New York": Point(40.6446, -73.7797)      # KJFK
}

In [62]:
start = datetime(2011, 1, 1)
end = datetime(2016, 12, 31)

In [63]:
def weekyearnum(dt):
    return dt.strftime("%Y%W")

def myweeyearknum(dt):
    offsetdt = dt + timedelta(days=+2) # Adding 2 days offset
    return weekyearnum(offsetdt)

In [73]:
list_of_dfs = []

for city_name, location in cities_config.items():
    
    data = Daily(location, start, end)
    data = data.fetch()
    
    data['rainy_day'] = (data['prcp'] > 0).astype(int)
    
    data['yearweek'] = data.index.to_series().apply(myweeyearknum)
    
    weekly_weather = data.groupby('yearweek').agg({
    'tavg': 'mean',  # Average Temp
    'tmin': 'min',   # Minimum Temp
    'tmax': 'max',   # Maximum Temp
    'prcp': 'sum',   # Total precipitation volume
    'snow': 'sum',   # Total snowfall
    'wspd': 'mean',  # Average wind speed
    'rainy_day': 'sum'  # Count of rainy days in the week
    })
    
    weekly_weather.rename(columns={'rainy_day': 'rain_days_count'}, inplace=True)

    weekly_weather['region'] = city_name
    
    weekly_weather = weekly_weather.reset_index()
    weekly_weather['yearweek'] = weekly_weather['yearweek'].astype(int)
    
    list_of_dfs.append(weekly_weather)

final_weather_df = pd.concat(list_of_dfs)
final_weather_df.to_csv('./weekly_weather_data_NYC_BOS_PHI.csv', index=False)

In [74]:
final_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 960 entries, 0 to 319
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   yearweek         960 non-null    int64  
 1   tavg             960 non-null    Float64
 2   tmin             960 non-null    Float64
 3   tmax             960 non-null    Float64
 4   prcp             960 non-null    Float64
 5   snow             960 non-null    Float64
 6   wspd             960 non-null    Float64
 7   rain_days_count  960 non-null    int64  
 8   region           960 non-null    object 
dtypes: Float64(6), int64(2), object(1)
memory usage: 80.6+ KB


In [70]:
final_weather_df.head()

,yearweek,tavg,tmin,tmax,prcp,snow,wspd,rain_days_count,region
0,201101,1.514286,-5.6,13.3,4.3,66.0,15.171429,2,Boston
1,201102,-2.357143,-7.8,1.7,43.0,157.0,23.142857,3,Boston
2,201103,-2.5,-10.0,4.4,47.4,265.0,16.928571,3,Boston
3,201104,-6.071429,-18.9,1.7,21.4,379.0,15.471429,3,Boston
4,201105,-3.642857,-8.9,2.8,33.2,470.0,14.771429,2,Boston


In [66]:
df_weekly_backup2 = df_weekly.copy()

In [75]:
df_weekly.merge(final_weather_df, on=['yearweek', 'region'], how='left')

,item,store_code,category_x,department,region,store,yearweek,sale,sell_price,daily_revenue,...,market_presence,sin_year,cos_year,tavg,tmin,tmax,prcp,snow,wspd,rain_days_count
0,ACCESORIES_1_004,BOS_1,ACCESORIES,ACCESORIES_1,Boston,South_End,201105.0,10,5.77,57.72,...,0,0.500000,0.866025,-3.642857,-8.9,2.8,33.2,470.0,14.771429,2
1,ACCESORIES_1_004,BOS_2,ACCESORIES,ACCESORIES_1,Boston,Roxbury,201105.0,15,5.77,86.58,...,0,0.500000,0.866025,-3.642857,-8.9,2.8,33.2,470.0,14.771429,2
2,ACCESORIES_1_004,NYC_2,ACCESORIES,ACCESORIES_1,New York,Harlem,201105.0,33,5.77,190.48,...,0,0.500000,0.866025,-1.2,-6.7,3.9,19.6,147.0,17.214286,3
3,ACCESORIES_1_004,NYC_4,ACCESORIES,ACCESORIES_1,New York,Brooklyn,201105.0,8,5.77,46.18,...,0,0.500000,0.866025,-1.2,-6.7,3.9,19.6,147.0,17.214286,3
4,ACCESORIES_1_004,PHI_1,ACCESORIES,ACCESORIES_1,Philadelphia,Midtown_Village,201105.0,6,5.77,34.63,...,0,0.500000,0.866025,-0.728571,-7.8,5.6,18.4,121.0,13.314286,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6721781,SUPERMARKET_3_827,NYC_3,SUPERMARKET,SUPERMARKET_3,New York,Tribeca,201617.0,16,1.20,19.20,...,114,0.866025,-0.500000,12.657143,7.8,21.7,4.8,0.0,15.885714,3
6721782,SUPERMARKET_3_827,NYC_4,SUPERMARKET,SUPERMARKET_3,New York,Brooklyn,201617.0,12,1.20,14.40,...,114,0.866025,-0.500000,12.657143,7.8,21.7,4.8,0.0,15.885714,3
6721783,SUPERMARKET_3_827,PHI_1,SUPERMARKET,SUPERMARKET_3,Philadelphia,Midtown_Village,201617.0,0,1.20,0.00,...,114,0.866025,-0.500000,14.4,7.2,30.0,9.9,0.0,14.4,3
6721784,SUPERMARKET_3_827,PHI_2,SUPERMARKET,SUPERMARKET_3,Philadelphia,Yorktown,201617.0,2,1.20,2.40,...,114,0.866025,-0.500000,14.4,7.2,30.0,9.9,0.0,14.4,3


### Añadimos modificacion de eventos

In [85]:
df_calendar.head()

,yearweek,date,Weekly_Event,Year,Quarter,Month,Day_month
0,201105.0,2011-01-29,No Event,2011,1,1,29
1,201106.0,2011-02-05,Superbowl,2011,1,2,5
2,201107.0,2011-02-12,Valentine's Day,2011,1,2,12
3,201108.0,2011-02-19,Presidents' Day,2011,1,2,19
4,201109.0,2011-02-26,No Event,2011,1,2,26


In [87]:
## Como ya tenemos los datos climáticos (nieve, viento, etc), añadir el 
# evento de "blizzard" y "hurricane" en la semana correspondiente sería redundante

clima_redundante = ["Blizzard", "Hurricane"] 

def limpiar_eventos_climaticos(texto_evento):
    for termino in clima_redundante:
        if termino in texto_evento:
            return "No Event"

    return texto_evento

# Aplicamos la limpieza al calendario
df_calendar['Weekly_Event_Clean'] = df_calendar['Weekly_Event'].apply(limpiar_eventos_climaticos)

In [ ]:
def crear_variable_anticipacion(df, event_column, string_event, max_weeks=8):
    """
    Creates a 'heat' feature (0 to max_weeks) indicating proximity to an event.
    df: DataFrame of the calendar (ordered)
    event_column: Name of the column with event texts (e.g., 'Weekly_Event_Clean')
    string_event: Text to search for (e.g., "Valentine's")
    max_weeks: How many weeks before to start counting (e.g., 4 or 8)
    """
    
    # identification of event occurrences (Boolean)
    is_event = df[event_column].str.contains(string_event, na=False, regex=False)
    
    # index series where event occurs
    event_indices = pd.Series(np.where(is_event, df.index, np.nan), index=df.index)
    
    # fill NaNs backwards with next valid event index
    next_event_index = event_indices.bfill()
    
    # Transform distance into "heat" (invert)
    distance = next_event_index - df.index
    heat_feature = distance.apply(lambda x: max(0, max_weeks - x) if not np.isnan(x) else 0)
    
    return heat_feature

In [92]:
df_calendar = df_calendar.sort_values('yearweek').reset_index(drop=True)

df_calendar['valentines_heat'] = crear_variable_anticipacion(df_calendar, 'Weekly_Event_Clean', "Valentine's Day", max_weeks=4)
df_calendar['xmas_heat'] = crear_variable_anticipacion(df_calendar, 'Weekly_Event_Clean', "Christmas", max_weeks=8)
df_calendar['superbowl_heat'] = crear_variable_anticipacion(df_calendar, 'Weekly_Event_Clean', "Superbowl", max_weeks=3)
df_calendar['thanksgiving_heat'] = crear_variable_anticipacion(df_calendar, 'Weekly_Event_Clean', "Thanksgiving", max_weeks=3)
df_calendar['july4_heat'] = crear_variable_anticipacion(df_calendar, 'Weekly_Event_Clean', "Independence Day", max_weeks=3)

# Verificación visual
print(df_calendar[['yearweek', 'Weekly_Event_Clean', 'valentines_heat', 'xmas_heat', 'superbowl_heat', 'thanksgiving_heat', 'july4_heat']].head(20))

    yearweek            Weekly_Event_Clean  valentines_heat  xmas_heat  \
0   201105.0                      No Event              2.0        0.0   
1   201106.0                     Superbowl              3.0        0.0   
2   201107.0               Valentine's Day              4.0        0.0   
3   201108.0               Presidents' Day              0.0        0.0   
4   201109.0                      No Event              0.0        0.0   
5   201110.0                      No Event              0.0        0.0   
6   201111.0                      No Event              0.0        0.0   
7   201112.0                      No Event              0.0        0.0   
8   201113.0                      No Event              0.0        0.0   
9   201114.0                      No Event              0.0        0.0   
10  201115.0                      No Event              0.0        0.0   
11  201116.0        Patriots' Day (Boston)              0.0        0.0   
12  201117.0                 Easter Su

In [93]:
shopping_keys = ["Black Friday", "Cyber Monday", "Christmas", "Valentine's Day"]
closed_keys = ["New Year's Day", "Thanksgiving", "Christmas", "Independence Day"]

In [94]:
def classify_event(event_string, keys):
    """Returns 1 if any key is found in the event string description."""
    if event_string == "No Event":
        return 0
    # Check if any key substring is present in the weekly event string
    for key in keys:
        if key in event_string:
            return 1
    return 0

In [95]:
df_calendar['is_shopping_event'] = df_calendar['Weekly_Event_Clean'].apply(lambda x: classify_event(x, shopping_keys))
df_calendar['is_holiday_closed'] = df_calendar['Weekly_Event_Clean'].apply(lambda x: classify_event(x, closed_keys))

In [96]:
df_calendar.head()

,yearweek,date,Weekly_Event,Year,Quarter,Month,Day_month,Weekly_Event_Clean,valentines_heat,xmas_heat,superbowl_heat,thanksgiving_heat,july4_heat,is_shopping_event,is_holiday_closed
0,201105.0,2011-01-29,No Event,2011,1,1,29,No Event,2.0,0.0,2.0,0.0,0.0,0,0
1,201106.0,2011-02-05,Superbowl,2011,1,2,5,Superbowl,3.0,0.0,3.0,0.0,0.0,0,0
2,201107.0,2011-02-12,Valentine's Day,2011,1,2,12,Valentine's Day,4.0,0.0,0.0,0.0,0.0,1,0
3,201108.0,2011-02-19,Presidents' Day,2011,1,2,19,Presidents' Day,0.0,0.0,0.0,0.0,0.0,0,0
4,201109.0,2011-02-26,No Event,2011,1,2,26,No Event,0.0,0.0,0.0,0.0,0.0,0,0


In [97]:
df_weekly_backup3 = df_weekly.copy()

In [98]:
### Hacemos merge del calendario con las nuevas features al df_weekly
df_weekly = df_weekly.merge(df_calendar[['yearweek', 'valentines_heat', 'xmas_heat', 'superbowl_heat',
                                        'thanksgiving_heat', 'july4_heat', 'is_shopping_event', 'is_holiday_closed']],
                            on='yearweek', how='left')

In [99]:
df_weekly.head()

,item,store_code,category_x,department,region,store,yearweek,sale,sell_price,daily_revenue,...,market_presence,sin_year,cos_year,valentines_heat,xmas_heat,superbowl_heat,thanksgiving_heat,july4_heat,is_shopping_event,is_holiday_closed
0,ACCESORIES_1_004,BOS_1,ACCESORIES,ACCESORIES_1,Boston,South_End,201105.0,10,5.77,57.72,...,0,0.5,0.866025,2.0,0.0,2.0,0.0,0.0,0,0
1,ACCESORIES_1_004,BOS_2,ACCESORIES,ACCESORIES_1,Boston,Roxbury,201105.0,15,5.77,86.58,...,0,0.5,0.866025,2.0,0.0,2.0,0.0,0.0,0,0
2,ACCESORIES_1_004,NYC_2,ACCESORIES,ACCESORIES_1,New York,Harlem,201105.0,33,5.77,190.48,...,0,0.5,0.866025,2.0,0.0,2.0,0.0,0.0,0,0
3,ACCESORIES_1_004,NYC_4,ACCESORIES,ACCESORIES_1,New York,Brooklyn,201105.0,8,5.77,46.18,...,0,0.5,0.866025,2.0,0.0,2.0,0.0,0.0,0,0
4,ACCESORIES_1_004,PHI_1,ACCESORIES,ACCESORIES_1,Philadelphia,Midtown_Village,201105.0,6,5.77,34.63,...,0,0.5,0.866025,2.0,0.0,2.0,0.0,0.0,0,0


In [ ]:
# Lags (Past values)
df_weekly.sort_values(by=['yearweek', 'item', 'store_code'],ascending=True, inplace=True)
# Previous week sales (t-1)
df_weekly['lag_1_sale'] = df_weekly.groupby(['item', 'store_code'])['sale'].shift(1)
df_weekly['lag_1_sellprice'] = df_weekly.groupby(['item', 'store_code'])['sell_price'].shift(1)

# Sales 4 weeks ago (approx 1 month)
df_weekly['lag_4_sale'] = df_weekly.groupby(['item', 'store_code'])['sale'].shift(4)
df_weekly['lag_4_sellprice'] = df_weekly.groupby(['item', 'store_code'])['sell_price'].shift(4)

# Sales 26 weeks ago (approx 6 months)
df_weekly['lag_26_sale'] = df_weekly.groupby(['item', 'store_code'])['sale'].shift(26)
df_weekly['lag_26_sellprice'] = df_weekly.groupby(['item', 'store_code'])['sell_price'].shift(26)

# Sales 52 weeks ago (approx 1 year)
df_weekly['lag_52_sale'] = df_weekly.groupby(['item', 'store_code'])['sale'].shift(52)
df_weekly['lag_52_sellprice'] = df_weekly.groupby(['item', 'store_code'])['sell_price'].shift(52)



In [ ]:
# Rolling Features
# 4-week moving average (shows trend without noise)
df_weekly['rolling_mean_4w_sale'] = df_weekly.groupby(['item', 'store_code'])['sale'].transform(lambda x: x.rolling(window=4).mean())
df_weekly['rolling_mean_4w_sellprice'] = df_weekly.groupby(['item', 'store_code'])['sell_price'].transform(lambda x: x.rolling(window=4).mean())


In [ ]:
# Variation / Detrending (Target for stationary models)
# "Diff": Absolute difference (Current - Previous)
df_weekly['sales_diff'] = df_weekly.groupby(['item', 'store_code'])['sale'].diff(1)
df_weekly['price_diff'] = df_weekly.groupby(['item', 'store_code'])['sell_price'].diff(1)

In [ ]:
df_calen

## 3. Particion

In [ ]:
week_number_val = 260
week_number_test = 274

df_validation = df_weekly[df_weekly.week_number.between(week_number_val, week_number_test - 1)]
df_test = df_weekly[df_weekly.week_number >= week_number_test]